In [ ]:
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator

drive.mount('/content/drive')

In [ ]:
## Visualising dataset

dataset = '/content/drive/MyDrive/Colab/train_val'

import matplotlib.pyplot as plt
import random
import cv2
import os

def extract_labels(filename):
    parts = filename.split('_')
    age = int(parts[0])
    gender = int(parts[1])
    gender_num = int(parts[1])
    gender = "male" if gender_num == 0 else "female"
    return age, gender

filenames = []
ages = []
genders = []

for fname in os.listdir(dataset):
    if fname.endswith('.jpg'):
        age, gender = extract_labels(fname)
        filenames.append(fname)
        ages.append(age)
        genders.append(gender)

sample_files = random.sample(os.listdir(dataset), 20)

plt.figure(figsize=(10,10))
for i, fname in enumerate(sample_files):
    img = plt.imread(os.path.join(dataset, fname))
    age, gender_label = extract_labels(fname)

    plt.subplot(4, 5, i + 1)
    plt.imshow(img)
    plt.title(f"{gender_label}, {age} yrs", fontsize=9)
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
## Pre-processing and data augmentation

import os
import random
import numpy as np
import tensorflow as tf

folder_train_val = '/content/drive/MyDrive/Colab/train_val'
all_image_files = [file for file in os.listdir(folder_train_val) if file.lower().endswith(('.jpg'))]

random.seed(0)
random.shuffle(all_image_files)

n_train_val = len(all_image_files)
train_end = int(n_train_val * 0.8)

train_image_files = all_image_files[:train_end]
val_image_files = all_image_files[train_end:]
print(f"Number of images for training: {len(train_image_files)}")
print(f"Number of images for validation: {len(val_image_files)}")

def load_imgs_lables(dataset_path,filenames):
  print('load all image data, age and gender labels...')
  images = []
  age_labels = []
  gender_labels = []
  for current_file_name in filenames:
    img = cv2.imread(os.path.join(dataset_path, current_file_name))
    img = img / 255.0
    labels = current_file_name.split('_')
    age_label = int(labels[0])
    gender_label = int(labels[1])
    age_labels.append(age_label)
    gender_labels.append(gender_label)
    images.append(img)

  images = np.array(images)
  age_labels = np.array(age_labels)
  gender_labels = np.array(gender_labels)
  return images, age_labels, gender_labels

train_images, train_age, train_gender = load_imgs_lables(folder_train_val,train_image_files)

val_images, val_age, val_gender = load_imgs_lables(folder_train_val,val_image_files)

from tensorflow.keras import layers, Sequential

data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.05),
    layers.RandomTranslation(0.05, 0.05)
])

In [ ]:
## Building network

import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, SpatialDropout2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

inputs = Input(shape=(128, 128, 3))

x = data_augmentation(inputs)

x = Conv2D(32, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = SpatialDropout2D(0.05)(x)

x = Conv2D(64, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = SpatialDropout2D(0.05)(x)

x = Conv2D(128, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = SpatialDropout2D(0.05)(x)

x = Conv2D(256, (3,3), padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = SpatialDropout2D(0.05)(x)

x = Flatten()(x)

x = Dense(256, activation='relu', kernel_initializer='he_normal')(x)

a = Dense(128, activation='relu')(x)
a = Dense(64, activation='relu')(a)
a = Dense(1, activation='linear',name='age_output')(a)

g = Dense(64, activation='relu')(x)
g = BatchNormalization()(g)
g = Dropout(0.1)(g)
g = Dense(1, activation='sigmoid',name='gender_output', kernel_initializer = 'glorot_normal')(g)

modelA = Model(inputs, [g, a])

modelA.summary()
from tensorflow.keras.utils import plot_model
plot_model(modelA, show_shapes=True,dpi=100)

In [ ]:
## Compiling and training network

from tensorflow.keras.losses import Huber, KLDivergence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint

optimizer = Adam(1e-4)

modelA.compile(optimizer = optimizer,
               loss = {
                   'gender_output': 'binary_crossentropy',
                   'age_output': tf.keras.losses.Huber(delta=2.0)
               },
               metrics = {
                   'gender_output': ['accuracy'],
                   'age_output': ['mae']
               })

callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8,
                                         patience=8, min_lr=1e-5, verbose=1)

es_callback = EarlyStopping( monitor='val_loss',
min_delta = 0.0005,
patience=25,
restore_best_weights=True)

history = modelA.fit(
    train_images,
    {'gender_output':train_gender, 'age_output':train_age},
    validation_data = (val_images, {'gender_output':val_gender, 'age_output':val_age}),
    epochs = 250,
    batch_size = 32,
    callbacks = [callback, es_callback],
    shuffle = True
)

folder_models = '/content/drive/MyDrive/Colab/'

modelA.save(folder_models+'age_gender_A.keras')

In [ ]:
## Learing curves

import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.plot(history.history['gender_output_loss'], label='Training Loss')
plt.plot(history.history['val_gender_output_loss'], label='Validation Loss')
plt.title('Gender Classification - Loss')
plt.xlabel('Epoch')
plt.ylabel('Binary Crossentropy Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(6,4))
plt.plot(history.history['gender_output_accuracy'], label='Training Accuracy')
plt.plot(history.history['val_gender_output_accuracy'], label='Validation Accuracy')
plt.title('Gender Classification - Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(6,4))
plt.plot(history.history['age_output_loss'], label='Training Loss')
plt.plot(history.history['val_age_output_loss'], label='Validation Loss')
plt.title('Age Estimation - Loss')
plt.xlabel('Epoch')
plt.ylabel('Huber Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(6,4))
plt.plot(history.history['age_output_mae'], label='Training MAE')
plt.plot(history.history['val_age_output_mae'], label='Validation MAE')
plt.title('Age Estimation - Mean Absolute Error')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.grid(True)
plt.show()